# DL2023 Assignment 3

VH: Textblocks that start with 'VH' means that I inserted that block.


---

VH: The structure of the helper files and images has to be of the following structure in order to run properly:
```bash
- code
    |- flickr8k_split (unziped)
    |- test_examples
    |- assignment3.ipynb
    |- evaluate.py
    |- get_loader.py
    |- model.py
    |- utils.py
```

VH: Model description

### Encoder
Type: Convolutional Neutral Network
Input: Images
Output: Image features

The EncoderCNN takes an image as an input, applies a pre-trained CNN model - ResNet-50 - with pretrained weights models.ResNet50_Weights.IMAGENET1K_V2 from the ImageNet. It does output the image features that can be further used by the captioning model (the decoder) for generating captions.

More precisly, in the 'forward' method, the input 'images' are passed through the pre-trained ResNet-50 model. The output features are then batch-normalized, passed through the ReLU activation function, and finally, dropout is applied to the features before returning them.

##### Changes compared to the original model:
- Using the pretrained weights IMAGENET1K_V2 of the resnet50 model to improve the accuracy.
- Added Batch-Normlization to speed up training time and improving the model's performance by normalizing the inputs to each layer.


### Decoder
Type: Recurrent Neural Network with Gated Recurrent Unit (GRU) architecture.
Input: Image features and captions
Output: Predicted word scores for each time step

The DecoderRNN class takes image features and captions as input and generates predicted word scores at each time step using the GRU architecture. These scores can be used for further steps like generating captions or computing the loss during training.

More precisly, in the 'forward' method, the input 'features' (image features) and 'captions' are passed through the embedding layer to get the embedded captions. Important here is to mention, that during training, the teacher forcing mechanism is used, ie the predictions of the previous step are not considered for the current step. The image features are concatenated with the embedded captions. Theses embeddings are then passed through the GRU layer, producing hidden states at each time step. Finally, the hidden states are passed through the linear layer to obtain the predicted word scores.

##### Changes compared to the original model:
- Using a different architecture such as the GRU network instead of the RNN network.

### Training Process
This training procedure trains the model over 301 epochs, using batches of size 256. The model is optimized using the Adam optimizer with a learning rate of 1e-4. The loss is computed using CrossEntropyLoss, and gradient clipping is applied to prevent unstable gradient updates. The procedure iterates through the dataset, updating the model's parameters to minimize the loss and improve the model's performance. However fine-tuning the CNN is not used. 

#### Changes compared to the original training process
- Gradient clipping is used to prevent exploding gradients and improve the stability of the training process.

---

VH: To load the python scripts, we need to add the folder of the python scripts to the system .

In [1]:
import sys
sys.path.append('code')

### Warning

The code for this baseline is adopted mainly from the above link:
https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/more_advanced/image_captioning


In [2]:
import os 
import json
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from utils import save_checkpoint, load_checkpoint, print_examples
from get_loader import get_loader, build_vocab
from get_loader import transform_data as transform
from model import CNNtoRNN
from evaluate import evaluate_dataset_new

### Define vocabulary

In [3]:
vocab = build_vocab("flickr8k_split/captions.txt", freq_threshold = 5)

### Define hyperparameters

In [4]:
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
load_model = False
save_model = True
train_CNN = False

# Hyperparameters
batch_size = 256
embed_size = 128
hidden_size = 128
vocab_size = len(vocab)
num_layers = 1
learning_rate = 1e-4
num_epochs = 301

cuda


In [5]:
run_name = 'baseline'
output_dir = os.path.join('checkpoints', run_name)
os.makedirs(output_dir , exist_ok= True)
step = 0

### Define train and test loaders

In [6]:
train_loader, dataset = get_loader(
    root_folder="flickr8k_split/train_images",
    annotation_file="flickr8k_split/train_captions.txt",
    transform=transform,
    vocab = vocab, # None
    num_workers=8,
    batch_size = batch_size
)


val_loader, _ = get_loader(
    root_folder="flickr8k_split/val_images",
    annotation_file="flickr8k_split/val_captions.txt",
    transform=transform,
    vocab = vocab, #dataset.vocab,
    num_workers=1,
    batch_size = 5,
    shuffle = False,
) 

### Define another train loder that is used to evaluate BLEU-1 score

In [7]:
train_loader_eval, _ = get_loader(
    root_folder="flickr8k_split/train_images",
    annotation_file="flickr8k_split/train_captions.txt",
    transform=transform,
    vocab = vocab, # None
    num_workers=1,
    batch_size = 5, # each image has 5 different GT captions, for each batch we will load only one image and corresponding captions
    shuffle = False,
)

### Define your model and optimizers. 

In [8]:
# initialize model, loss etc
model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=vocab.stoi["<PAD>"]) # it will ignore 'PAD' words 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Only finetune the CNN
for name, param in model.encoderCNN.resnet50.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN

In [10]:
if load_model:
    step = load_checkpoint(torch.load('SPECIFIY_PATH.pth.tar'), model, optimizer)

### Training loop

In [11]:
model.train()
for epoch in range(num_epochs):
    # Uncomment the line below to see a couple of test cases
    if epoch %50 ==0: # modify for your case
        print_examples(model, device, dataset)
    
    if epoch % 50 ==0: # modify for your case
        # calculate BLUE score on the validation set
        blue_score_val = evaluate_dataset_new(model, val_loader, vocab, device)
        print('BLUE SCORES validation ', epoch, blue_score_val)
        # calculate BLUE score on the training set
        blue_score_train = evaluate_dataset_new(model, train_loader_eval, vocab, device)
        print('BLUE SCORES TRAIN', epoch, blue_score_train)
        
        # logging
        log_stats = {'BLUE-1-VAL': blue_score_val,
                     'BLUE-1-TRAIN': blue_score_train,
                    'epoch': epoch}
        f = open(os.path.join(output_dir, "log_blue.txt"), "a+")
        f.write(json.dumps(log_stats) + "\n")
        f.close()

        # save model every epoch - modify it you do not want to save it every epoch
        if save_model:
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "step": step,
            }
            save_checkpoint(checkpoint, filename = os.path.join(output_dir, 'checkpoints_'+str(epoch)+'.pth.tar'))
    
    for idx, (imgs, captions) in tqdm(
        enumerate(train_loader), total=len(train_loader)
    ):

        imgs = imgs.to(device)
        captions = captions.to(device)
        outputs = model(imgs, captions[:-1])
        loss = criterion(
            outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
        )

        step += 1

        optimizer.zero_grad()
        loss.backward(loss)
        # use clip grad if needed
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 10.)
        optimizer.step()
        # logging
        log_stats = {'loss': loss.item(),
                    'epoch': epoch,
                    'step': step}
        f = open(os.path.join(output_dir, "log_loss.txt"), "a+")
        f.write(json.dumps(log_stats) + "\n")
        f.close()
    print("Training loss", loss.item())

Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: overhead traffic brother breaks parlor handbag carts beside hugs struggle backward surfer bears wait shirtless video sparklers stomach lockers unseen earring along face hovering papers nails university league bird saber chased heavy eat rails cowboy cowboy stars carriage desert lunges wrestlers shallow piece well piece stroller papers downtown officer chased
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: mohawk return groom lean tall paved marx bending shirt tries tackled bag motorbike rural higher wet hiking overlooking band sell dad coat without posters officer form hold overlooks hamburgers masks athlete sort butter butter pants skirts hear cream silly skateboarder there wakeboard obama shepherd breaks snap hay armenian piano sun
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: coach dim self sparklers canoe squats bandanna surfing cowboy clad baseman unique crouched crouched wai

BLUE SCORES validation  0 0.5922767116796966


BLUE SCORES TRAIN 0 0.7109332422297603
=> Saving checkpoint


100%|██████████| 139/139 [00:56<00:00,  2.47it/s]


Training loss 6.267791271209717


...

Training loss 2.1832804679870605
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a brown dog is running through the water . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a little girl in a pink shirt is playing with a ball in a park . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man in a red shirt and a red shirt is standing on a motorcycle . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a man is standing on a dock . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man in a red jacket is standing on a mountain . <EOS>


BLUE SCORES validation  300 65.66297116545734


BLUE SCORES TRAIN 300 70.03197163048456
=> Saving checkpoint


100%|██████████| 139/139 [00:51<00:00,  2.70it/s]

Training loss 2.2188913822174072


VH: Hence, the training blue score is 70.03 and the validation blue score is 65.66.

VH: Run the model for the test set to generate the captions.

In [28]:
import csv
from PIL import Image
import fnmatch

# Open the output file in write mode
with open('output.txt', mode='w') as file:
    model.eval()

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['image', 'caption'])
    directory = "flickr8k_split/test_images"
    test_dir_size = count = len(fnmatch.filter(os.listdir(directory), '*.*'))

    # Loop through the test images
    for i, image_name in enumerate(os.listdir(directory)):
        # Load the test image
        image_path = os.path.join(directory, image_name)
        test_img = transform(Image.open(image_path).convert("RGB")).unsqueeze(0)

        # Generate the caption using the model
        caption = " ".join(model.caption_image(test_img.to(device), dataset.vocab))
        caption = caption[len('<SOS> '): -len(' <EOS>')].strip().capitalize()

        # Write the image path and caption to the CSV file
        writer.writerow([image_name, caption])

        # Print the progress
        print(f'{i+1}/{test_dir_size}: {image_path} => {caption}')

1/500: flickr8k_split/test_images\130211457_be3f6b335d.jpg => A woman in a red shirt and a white shirt is standing on a sidewalk .
2/500: flickr8k_split/test_images\131632409_4de0d4e710.jpg => A girl in a pink bathing suit is standing on a beach .
3/500: flickr8k_split/test_images\132489044_3be606baf7.jpg => A woman in a white shirt and a white shirt is sitting on a bench .
4/500: flickr8k_split/test_images\133189853_811de6ab2a.jpg => A man in a black shirt and sunglasses is standing in front of a crowd .
5/500: flickr8k_split/test_images\133905560_9d012b47f3.jpg => A white dog is running through the snow .
6/500: flickr8k_split/test_images\134724228_30408cd77f.jpg => A man and a woman are standing in the middle of a large body of water .
7/500: flickr8k_split/test_images\134894450_dadea45d65.jpg => A boy in a red shirt is jumping off a swing .
8/500: flickr8k_split/test_images\135235570_5698072cd4.jpg => A man in a black shirt and white shorts is standing on a sidewalk with a woman in